In [1]:
%run "libraries.ipynb"

from IPython.display import display, HTML

from bs4 import BeautifulSoup

## storing the original namespace

In [2]:
pages = codecs.open("data/pagenames.txt","r", "utf-8-sig").readlines()
pages = map(lambda x: x.strip(), pages)

## finding wikipedia links in the page content

In [3]:
hyperlinks_graph = nx.DiGraph()

def find_hyperlinks(page):
    hyperlinks = []
    
    with codecs.open("data/pages/%s.json" % (page), "r", "utf-8-sig") as f:
        j = json.load(f)
        content = j["query"]["pages"][j["query"]["pages"].keys()[0]]
        content = content["revisions"][0]["*"]

    content = BeautifulSoup(content, 'html.parser') 
    
    hyperlinks = content.find_all('a')
        
    return hyperlinks

for p in pages:
    hyperlinks = find_hyperlinks(p)
    
    # keep only title of the link
    hyperlinks = map(lambda x: x.get("title"), hyperlinks)
    hyperlinks = list(set(hyperlinks))

    # keep only linked pages that are inside the initial domain
    intradomain_pages = set(hyperlinks) & set(pages)
    extradomain_pages = set(hyperlinks) - set(pages)

#     print len( hyperlinks )
#     print len( intradomain_pages )
#     print len( extradomain_pages )
    
    for target in intradomain_pages:
        hyperlinks_graph.add_edge(p, target)

print "nodes: %s" % len(hyperlinks_graph.nodes())
print "edges: %s" % len(hyperlinks_graph.edges())

nodes: 300
edges: 3781


## storing the result graph

In [4]:
nx.write_gexf(hyperlinks_graph, "data/hyperlinks.gexf")

## community detection (louvain)

In [5]:
import community
partitions = community.best_partition(hyperlinks_graph.to_undirected())

In [6]:
def print_groups(communities):
    html = "<table>"

    for c, ps in communities.iteritems():
        html += "<tr><td style=\"width: 100px; text-align: center; \"><h3>group %s</h3></td><td>" % (c)
        html += ", ".join(map(lambda x: u"<a href=\"http://en.wikipedia.org/wiki/{0}\" target=\"_blank\">{0}</a>".format(x), ps))
        html += "</td></tr>"
    
    html += "</table>"

    display(HTML(html))
        
communities = {}
for k, v in partitions.iteritems():
    communities.setdefault(v, []).append(k)


print_groups(communities)